In [1]:
import pandas as pd
import datetime as dt
from logger_settings import logger
import phase as ph

pd.options.display.max_colwidth = 10
pd.options.display.max_rows = None

In [2]:
ticks_columns = {
    'last_price': float,
    'last_traded_quantity': int,
    'total_buy_quantity': int,
    'total_sell_quantity': int,
    'last_trade_time': 'datetime64[ns]',
    'oi': int,
}
tdf = pd.read_json('ticks-formatted.json', dtype=ticks_columns, convert_dates={'last_trade_time': '%Y-%m-%d %H:%M:%S'})
columns_to_drop = set(tdf.columns).difference(ticks_columns.keys())
tdf.drop(columns=columns_to_drop, inplace=True)
tdf.set_index('last_trade_time', inplace=True, drop=False)


class Settings:
    MIN_UP_PC_CHG = 0.5 / 100 # .5%
    MIN_DOWN_PC_CHG = -0.5 / 100 # -.5%
    BREACH_PC = 5 / 100 # 5% Percentage of continuation phase gains required to breach the trend
    LAST_TICKS_BREACH = {ph.Direction.UP: 5, ph.Direction.DOWN: 5}
    QUANTITY = 50
    CONFIRM_TICKS = 5 # 5 Seconds

In [3]:
class ContinuationPhase(ph.Phase):
    def __init__(self, ticks, pm):
        self.phase_type = ph.Phase.TYPE_CONT
        self.t_start = ticks[0]
        self.t_end = ticks[-1]
        self.status = self.STATUS_INITIATED
        self.min_pc = None
        self.direction = None
        self.pm = pm
        self.last_processed_tid = ticks[-1].id
        self.started_at = None
        self.terminated_at = None
        self.hard_retraced_at = None
        self.soft_retraced_at = None
        self.last_5sec = ph.Candle(ticks, confirm_ticks=self.pm.settings.CONFIRM_TICKS)
        self.second_last_5sec = ph.Candle([], confirm_ticks=self.pm.settings.CONFIRM_TICKS)

    def start_phase(self):
        if self.t_start.id == 12068:
            print(f"{self.second_last_5sec}, l: {self.last_5sec}")
        if self.second_last_5sec.period < Settings.CONFIRM_TICKS or len(self.last_5sec) == 0:
            if self.t_start.id == 12068:
                print(f"f1: {self.second_last_5sec}, l: {self.last_5sec}")
            return False
        if self.second_last_5sec.IS_RED != self.last_5sec.IS_RED:
            raise ph.PhaseStartFailed(f'failed to start phase at: {self.last_5sec.end_id}')
        is_red_breach = self.second_last_5sec.IS_RED and self.last_5sec.CLOSE >= self.second_last_5sec.CLOSE
        is_green_breach = (not self.second_last_5sec.IS_RED) and self.last_5sec.CLOSE <= self.second_last_5sec.CLOSE
        if is_red_breach or is_green_breach:
            raise ph.PhaseStartFailed(f'failed to start phase at: {self.last_5sec.end_id}')
        if self.t_start.id == 12068:
            print(f"s1: {self.second_last_5sec}, l: {self.last_5sec}")
        price_change = self.last_5sec.CLOSE - self.second_last_5sec.OPEN
        self.min_pc = Settings.MIN_UP_PC_CHG if price_change >= 0 else Settings.MIN_DOWN_PC_CHG
        self.direction = ph.Direction.UP if price_change >= 0 else ph.Direction.DOWN
        self.status = self.STATUS_STARTED
        self.started_at = self.last_5sec[-1]
        return True

    def process(self, tick):
        """
        if it breaks previous open or previous nth tick
        """
        if tick.id <= self.last_processed_tid:
            return
        self.last_processed_tid = tick.id
        self.update_last_nsec(tick)
        self.t_end = tick
        if self.status == self.STATUS_INITIATED:
            has_started = self.start_phase()
            if not has_started:
                return
            self.pm.on_phase_start(self, tick)
            return
        elif self.status == self.STATUS_TERM or self.status == self.STATUS_HARD_RETR:
            return
        change = self.last_5sec.CLOSE - self.second_last_5sec.OPEN
        open_change = self.last_5sec.CLOSE - self.t_start.last_price
        last_5sec_breach = (self.direction == ph.Direction.UP and change < 0) or (self.direction == ph.Direction.DOWN and change > 0)
        open_breach = (self.direction == ph.Direction.UP and open_change < 0) or (self.direction == ph.Direction.DOWN and open_change > 0)
        if last_5sec_breach or open_breach:
            self.status = self.STATUS_HARD_RETR
            self.hard_retraced_at = tick
            raise ph.PhaseHardRetraceException(f"Continuation Phase {self} hard retracing at: {tick}")
        self.status = self.STATUS_ONG

In [4]:
class MyPhaseManager(ph.PhaseManager):
    def __init__(self, instrument, settings):
        super().__init__(instrument, settings)

    def process(self, tick):
        if self.current_phase is None:
            # First call
            self.current_phase = ContinuationPhase(ticks=[tick], pm=self)
            return
        # Has existing phase
        try:
            self.current_phase.process(tick)
            self.on_ongoing(self.current_phase, tick)
        except ph.PhaseSoftRetraceException:
            self.on_soft_retracel(self.current_phase, tick)
        except ph.PhaseHardRetraceException:
            self.on_hard_retracel(self.current_phase, tick)
            self.ps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except ph.PhaseTerminatedException:
            self.on_hard_retracel(self.current_phase, tick)
            self.on_termination(self.current_phase, tick)
            self.aps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except ph.PhaseStartFailed:
            logger.info(f"phase start failed at: {tick}, phase: {self.current_phase}")
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        # self.process_ps(tick)

    def process_ps(self, tick):
        # Process for process stack
        mark_for_removal = []
        for i in range(len(self.ps)):
            phase = self.ps[i]
            try:
                phase.process(tick)
            except ph.PhaseSoftRetraceException:
                # logger.info(f"soft retracel of {phase}")
                pass
            except ph.PhaseHardRetraceException:
                pass
            except ph.PhaseTerminatedException:
                mark_for_removal.append(i)
                self.aps.append(phase)
        mark_for_removal.reverse()
        for mark in mark_for_removal:
            del self.ps[mark]

    def on_phase_start(self, phase, tick):
        self.current_order = ph.Order(type=ph.Order.TYPE_BUY, limit_price=tick.last_price, quantity=Settings.QUANTITY)

    def on_hard_retracel(self, phase, tick):
        if phase.direction == ph.Direction.UP:
            if self.current_order is not None:
                self.current_order.square_off(tick.last_price)
                self.closed_orders.append(ph.PhaseOrder(phase, self.current_order))
                self.current_order = None
            logger.info(f"hard retracel {phase}, Up change %: {phase.started_at.last_price - phase.hard_retraced_at.last_price}")
        else:
            logger.info(f"hard retracel {phase}, Down change %: {phase.started_at.last_price - phase.hard_retraced_at.last_price}")

instrument = ph.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")
pm = MyPhaseManager(instrument=instrument, settings=Settings)
# for i in range(70):
for i in range(tdf.shape[0]):
    pm.next(tdf.iloc[i].to_dict())

print(pm.current_phase)
# print(pm.ps)
# print(pm.aps)

2024-03-12 07:04:39,751 - INFO - 235263041.py:28 - [140514618705152] - 
phase start failed at: Tick: 25925 -2.91, phase: Continuation: None, INITIATED from Tick: 25918 0.0 to Tick: 25925 -2.91, confirmation:None terminated at: None, hard retracel: None, soft: None
2024-03-12 07:04:39,754 - INFO - 235263041.py:28 - [140514618705152] - 
phase start failed at: Tick: 25931 -1.74, phase: Continuation: None, INITIATED from Tick: 25920 0.58 to Tick: 25931 -1.74, confirmation:None terminated at: None, hard retracel: None, soft: None
2024-03-12 07:04:39,766 - INFO - 235263041.py:61 - [140514618705152] - 
hard retracel Continuation: up, HARD RETRACING from Tick: 25926 -3.49 to Tick: 25955 0.58, confirmation:Tick: 25918 0.0 terminated at: None, hard retracel: Tick: 25955 0.58, soft: None, Up change %: -0.5813953488372176
2024-03-12 07:04:39,769 - INFO - 235263041.py:28 - [140514618705152] - 
phase start failed at: Tick: 25962 -2.33, phase: Continuation: None, INITIATED from Tick: 25950 2.91 to Ti

Continuation: None, INITIATED from Tick: 30316 54.65 to Tick: 30326 55.23, confirmation:None terminated at: None, hard retracel: None, soft: None


In [16]:
pm.ps[8]

Continuation: up, HARD RETRACING from Tick: 12143 -8.14 to Tick: 12156 -9.88, confirmation:Tick: 12155 -8.72 terminated at: None, hard retracel: Tick: 12156 -9.88, soft: None

In [10]:
profits = [po.order.pnl for po in pm.closed_orders]
# profits = [round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2) for ph in pm.aps if ph.direction == ph.Direction.UP]
# profits = [round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2) for ph in pm.ps if ph.direction == ph.Direction.UP]
print(sum(profits))
print(len(profits))

-7.519999999999999
143


In [5]:
# Losses
# multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders if po.order.pnl <= 0]
# multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders if po.order.pnl <= 0]
# texts = [str(po.order.pnl) for po in pm.closed_orders if po.order.pnl <= 0]
# subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

# All
from bokeh.models import ColumnDataSource

multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders]
multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders]
texts = [str(po.order.pnl) for po in pm.closed_orders]
subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

subplot = ColumnDataSource(subplot)

phase_in_id = [phase.t_start.id for phase in pm.ps]
phase_in_price = [phase.t_start.last_price for phase in pm.ps]
phase_start_plot = ColumnDataSource({'x': phase_in_id, 'y': phase_in_price})

p_end_id = [phase.t_end.id for phase in pm.ps]
p_end_price = [phase.t_end.last_price for phase in pm.ps]
phase_end_plot = ColumnDataSource({'x': p_end_id, 'y': p_end_price})

# phase_plot.data

In [6]:
import utils as ut
# ut.bokeh_plot(x=list(range(tdf.shape[0])), y=tdf.last_price, x_label="timestamp", y_label="price", plot='line')
ut.bokeh_plot(x=[m.id for m in pm.ticks], y=[tick.last_price for tick in pm.ticks], x_label="timestamp", y_label="price", plot='line', multi_plots=subplot, subplots=[phase_start_plot, phase_end_plot])
# tdf

Loading BokehJS ...

In [9]:
multi_y = [[ph.started_at.last_price, ph.hard_retraced_at.last_price] for ph in pm.aps if ph.direction == ph.Direction.UP]
multi_x = [[ph.started_at.id, ph.hard_retraced_at.id] for ph in pm.aps if ph.direction == ph.Direction.UP]
texts = [str(round(ph.hard_retraced_at.last_price - ph.started_at.last_price, 2)) for ph in pm.aps if ph.direction == ph.Direction.UP]
subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

from bokeh.models import ColumnDataSource
subplot = ColumnDataSource(subplot)